In [1]:
import pandas as pd
import requests

In [2]:
df = pd.DataFrame(columns=['year', 'voted_count', 'registered_count', 'voted_pct'])

### Add in manually calculated 2007-15 results 

In [3]:
df.loc[len(df.index)] = ['2007', 2425, 15008, 100 * (2425/15008)]
df.loc[len(df.index)] = ['2008', 11459, 19850, 100 * (11459/19850)]
df.loc[len(df.index)] = ['2009', 1894, 17570, 100 * (1894/17570)]
df.loc[len(df.index)] = ['2010', 4718, 18604, 100 * (4718/18604)]
df.loc[len(df.index)] = ['2011', 1886, 17840, 100 * (1886/17840)]
df.loc[len(df.index)] = ['2012', 10082, 20689, 100 * (10082/20689)]
df.loc[len(df.index)] = ['2013', 1432, 14565, 100 * (1432/14565)]
df.loc[len(df.index)] = ['2014', 2980, 18049, 100 * (2980/18049)]
df.loc[len(df.index)] = ['2015', 1578, 16492, 100 * (1578/16492)]

### Scrape 2016-21 results from county system

In [4]:
ELECTIONS = {
    '2016': 'https://results.enr.clarityelections.com/PA/Allegheny/63905/193472/json/status.json',
    '2017': 'https://results.enr.clarityelections.com/PA/Allegheny/71801/193474/json/status.json',
    '2018': 'https://results.enr.clarityelections.com/PA/Allegheny/92253/222754/json/status.json',
    '2019': 'https://results.enr.clarityelections.com//PA/Allegheny/98378/258755/json/status.json',
    '2020': 'https://results.enr.clarityelections.com//PA/Allegheny/106267/272321/json/status.json',
    '2021': 'https://results.enr.clarityelections.com/PA/Allegheny/111176/285575/json/status.json',
    '2022': 'https://results.enr.clarityelections.com/PA/Allegheny/115752/311585/json/status.json'
}

In [5]:
for election_year in ELECTIONS:
    r = requests.get(ELECTIONS[election_year], headers={
        'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/106.0.0.0 Safari/537.36'
      }).json()
    
    precinct_names = [ precinct for precinct in r['P'] if 'Pittsburgh Ward 4' in precinct or 'PITTSBURGH WARD 4' in precinct ]
    precinct_indices = [ r['P'].index(precinct_name) for precinct_name in precinct_names ]

    voted = 0
    registered = 0

    for precinct_index in precinct_indices:
        voted += r['B'][precinct_index]
        registered += r['R'][precinct_index]
        
        if election_year == '2022':
            print([r['P'][precinct_index], voted, registered, 100 * (voted/registered)])

    df.loc[len(df.index)] = [election_year, voted, registered, 100 * (voted/registered)]

['Pittsburgh Ward 4 Dist 1', 43, 227, 18.94273127753304]
['Pittsburgh Ward 4 Dist 2', 331, 1023, 32.355816226783965]
['Pittsburgh Ward 4 Dist 3', 462, 1349, 34.24759080800593]
['Pittsburgh Ward 4 Dist 4', 492, 1589, 30.962869729389553]
['Pittsburgh Ward 4 Dist 5', 795, 2560, 31.0546875]
['Pittsburgh Ward 4 Dist 6', 979, 3307, 29.603870577562745]
['Pittsburgh Ward 4 Dist 7', 1683, 5577, 30.17751479289941]
['Pittsburgh Ward 4 Dist 8', 2366, 8187, 28.899474777085622]
['Pittsburgh Ward 4 Dist 9', 2790, 9142, 30.518486108072633]
['Pittsburgh Ward 4 Dist 10', 3035, 9788, 31.007355946056396]
['Pittsburgh Ward 4 Dist 11', 3429, 10371, 33.06334972519526]
['Pittsburgh Ward 4 Dist 12', 3720, 11055, 33.649932157394844]
['Pittsburgh Ward 4 Dist 13', 3931, 11717, 33.54954339848084]
['Pittsburgh Ward 4 Dist 14', 4380, 13216, 33.14164648910412]
['Pittsburgh Ward 4 Dist 15', 4728, 14263, 33.148706443244755]
['Pittsburgh Ward 4 Dist 16', 4827, 15762, 30.62428625808907]
['Pittsburgh Ward 4 Dist 17', 5138

### Output results

In [6]:
df

,year,voted_count,registered_count,voted_pct
0,2007,2425,15008,16.158049
1,2008,11459,19850,57.727960
2,2009,1894,17570,10.779738
3,2010,4718,18604,25.360138
4,2011,1886,17840,10.571749
5,2012,10082,20689,48.731210
6,2013,1432,14565,9.831789
7,2014,2980,18049,16.510610
8,2015,1578,16492,9.568276
9,2016,11340,21142,53.637310


In [7]:
df[['year', 'voted_pct']].to_csv('pgh-ward-4.csv', index=False)